In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

## GET INDEKS LINK

In [8]:
response = requests.get("https://index.sindonews.com/index/0?t=2018-07-30")
    # Extract HTML texts contained in Response object: html
html = response.text
    # Create a BeautifulSoup object from the HTML: soup
soup = BeautifulSoup(html, "html5lib")
links = []

In [311]:
def getIndeksLink(links, page, cat_link, offset=0, date=datetime.strftime(datetime.today(), '%Y-%m-%d')):
    """
    Untuk mengambil seluruh url 
    link pada indeks category tertentu
    date format : YYYY/mm/dd
    """
    print("page ", page)
    url = "https://index.sindonews.com/index/"+ str(cat_link)+ "/" + str(offset)+ "?t="+ date
    print(url)
    
    # Make the request and create the response object: response
    response = requests.get(url)
    # Extract HTML texts contained in Response object: html
    html = response.text
    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")
    
    contentDiv = soup.find('div', class_="indeks-news")
    if contentDiv:
        for post in contentDiv.findAll('div', class_="indeks-title"):
            link = [post.find('a', href=True)['href'], ""]
            links.append(link)
        
    el_page = soup.find('div', class_="pagination")
    if el_page:
        max_page = int(soup.find('div', class_="pagination").findAll('a')[-2]['data-ci-pagination-page'])
            
        if page < max_page:
            links = getIndeksLink(links, page+1, cat_link, offset+10, date)
        
    return links

In [313]:
link = "https://index.sindonews.com/index/0/0?t=2018-07-30"
links = []

In [314]:
links = getIndeksLink([], 1, 0, 0, '2018-07-30')

page  1
https://index.sindonews.com/index/0/0?t=2018-07-30
page  2
https://index.sindonews.com/index/0/10?t=2018-07-30
page  3
https://index.sindonews.com/index/0/20?t=2018-07-30
page  4
https://index.sindonews.com/index/0/30?t=2018-07-30
page  5
https://index.sindonews.com/index/0/40?t=2018-07-30
page  6
https://index.sindonews.com/index/0/50?t=2018-07-30
page  7
https://index.sindonews.com/index/0/60?t=2018-07-30
page  8
https://index.sindonews.com/index/0/70?t=2018-07-30
page  9
https://index.sindonews.com/index/0/80?t=2018-07-30
page  10
https://index.sindonews.com/index/0/90?t=2018-07-30
page  11
https://index.sindonews.com/index/0/100?t=2018-07-30
page  12
https://index.sindonews.com/index/0/110?t=2018-07-30
page  13
https://index.sindonews.com/index/0/120?t=2018-07-30
page  14
https://index.sindonews.com/index/0/130?t=2018-07-30
page  15
https://index.sindonews.com/index/0/140?t=2018-07-30
page  16
https://index.sindonews.com/index/0/150?t=2018-07-30
page  17
https://index.sindo

In [299]:
# links

## GET DETAIL BERITA

In [217]:
response = requests.get("https://www.liputan6.com/news/read/3605072/pkb-mayoritas-ulama-ingin-jokowi-dua-periode")
    # Extract HTML texts contained in Response object: html
html = response.text
    # Create a BeautifulSoup object from the HTML: soup
soup = BeautifulSoup(html, "html5lib")

In [314]:
def getDetailBerita(links):
    all_articles = []
    for link in links:
        articles = {}
        #link
        url = link[0]
        response = requests.get(url)
        html = response.text
        # Create a BeautifulSoup object from the HTML: soup
        soup = BeautifulSoup(html, "html5lib")

        #extract subcategory from breadcrumb
        bc = soup.find('ul', class_="read-page--breadcrumb")
        if not bc:
            continue
        cat = bc.findAll('a')[-2].text
        sub = bc.findAll('a')[-1].text
            
        #articleid
        articleid = int(soup.find('article', class_="hentry main read-page--core-article")["data-article-id"])
        articles['id'] = articleid
        
        #category
        articles['category'] = cat
        articles['subcategory'] = sub
        
        #article_url
        articles['url'] = url        

        #article
        article = soup.findAll("div", class_="article-content-body__item-content")

        #extract date
        pubdate = soup.find('p', class_="read-page--header--author__datetime-wrapper").find('time').text
        pubdate = pubdate.strip(' \t\n\r')
        pubdate = pubdate.replace(' WIB','')
        articles['pubdate']=datetime.strftime(datetime.strptime(pubdate, "%d %b %Y, %H:%M"), "%Y-%m-%d %H:%M:%S")
        articles['pubdate']

        #extract author
        author = soup.find('a', class_="read-page--header--author__link url fn").find('span', class_="read-page--header--author__name fn").text
        articles['author'] = author

        #extract title
        title = soup.find('header', class_="read-page--header").find('h1').text
        if ("foto" in title.lower()) or  "video" in title.lower(): 
            continue
        articles['title'] = title   

        #source
        articles['source'] = 'Liputan6'

        #extract comments count
        comments = soup.find('li', class_="read-page--social-share__list-item js-social-share-comment").find('a')
        comments = int(comments.find('span', class_="read-page--social-share__comment-total").text)
        articles['comments'] = comments

        #extract tags
        tags = soup.findAll('span', class_="tags--snippet__name")
        tags = ','.join([x.text for x in tags])
        articles['tags'] = tags

        #extract images
        image = soup.find('picture', class_="read-page--photo-gallery--item__picture").find('img')['src']
        articles['image'] = image
        content = ''
        for art in article:
            #hapus link sisip
            for link in art.findAll('div'):
                link.decompose()

            #hapus figcaption
            for capt in art.findAll('figcaption'):
                capt.decompose()
            
            #extract content
            detail = BeautifulSoup(art.decode_contents().replace('<br/>', ' '), "html5lib")
            content += re.sub(r'\n|\t|\b|\r','',detail.text)
            
        #articles['content'] = re.sub('google*','', content).strip(' ')
        articles['content'] = content
        #print('memasukkan berita id ', articles['id'])
        all_articles.append(articles)
    return all_articles

In [76]:
links = getIndeksLink([], 1, 'news', 'news', '2018/07/30')

TypeError: getIndeksLink() takes from 2 to 3 positional arguments but 5 were given

In [75]:
# links

In [315]:
details = getDetailBerita(links[0:3])
details

[{'author': 'Luqman Rimadi',
  'category': 'News',
  'comments': 6,
  'content': 'Liputan6.com, Jakarta - Ijtimak Ulama yang tergabung dalam Gerakan Nasional Pembela Fatwa (GNPF) Ulama menyatakan dukungan kepada Prabowo Subianto di Pilpres 2019. Wakil Sekjen DPP Partai Kebangkitan Bangsa (PKB) Faisol reza menilai dukungan itu hanya klaim semata.Menurut dia, kebanyakan ulama yang ada di Indonesia masih menginginkan Presiden Jokowi untuk kembali memimpin Indonesia."Ijtimak ulama itu kan hanya mengikat satu kelompok saja, mayoritas ulama masih menginginkan Jokowi memimpin Indonesia untuk periode kedua," ujar Faiso Reza kepada wartawan di Jakarta, Senin (30/7/18).Menurut Faisol, seharusnya nama ulama tidak dibawa ke dalam pertarungan Pilpres 2019. Namun, Ijtimak GNPF yang memulai membawa nama ulama dalam gelanggang politik."Sedih karena 2019 akan ada pertempuran antar ulama. Tapi, tidak ada jalan mundur. Harus dihadapi," ujar dia.\xa0Faisol menambahkan, kedekatan Jokowi dengan para ulama s

In [316]:
df = pd.DataFrame(details)

In [317]:
df

,author,category,comments,content,id,image,pubdate,source,subcategory,tags,title,url
0,Luqman Rimadi,News,6,"Liputan6.com, Jakarta - Ijtimak Ulama yang ter...",3605072,https://cdn-production-assets-kly.akamaized.ne...,2018-07-30 23:15:00,Liputan6,Politik,"Jokowi,Ijtimak Ulama",PKB: Mayoritas Ulama Ingin Jokowi Dua Periode,https://www.liputan6.com/news/read/3605072/pkb...
1,Fachrur Rozie,News,0,"Liputan6.com, Jakarta - Penyidik Komisi Pember...",3605065,https://cdn-production-assets-kly.akamaized.ne...,2018-07-30 22:34:00,Liputan6,Peristiwa,"Korupsi Purbalingga,Bupati Purbalingga,Tasdi",4 Tersangka Suap Pembangunan Purbalingga Islam...,https://www.liputan6.com/news/read/3605065/4-t...
